In [1]:
# Script to read books from https://www.libreriainternacional.com/Libros

import requests
from requests.auth import HTTPProxyAuth
import re

import pandas as pd
import json
import random

import traceback

import time

from bs4 import BeautifulSoup

from pathlib import Path
import concurrent.futures

import datetime

current_proxy = 0

def get_proxy():
    global current_proxy
    i = current_proxy
    current_proxy = (current_proxy+1) % len(proxylist)
    
    # print('Proxy used: ', i)
    # proxies = {"http":proxylist['ip'][i]+':'+str(proxylist['port'][i]), "https":proxylist['ip'][i]+':'+str(proxylist['port'][i])}
    proxies = {"http": proxylist['ip'][i] + ':' + str(proxylist['port_http'][i])}
    auth = HTTPProxyAuth(proxylist['login'][i], proxylist['password'][i])

    return proxies, auth

proxylist = pd.read_csv('proxylist.csv', sep='\t')

In [3]:
# Dataframe for all books
all_books = pd.DataFrame()
endpoint = "https://www.libreriainternacional.com/Libros/GetBooks"
skip = 0

try:
    i = 0
    while i<1: #True:
        i += 1

        body = {
          "AlphaFilterId": -1,
          "LanguageId": -1,
          "authorId": -1,
          "editorialId": -1,
          "skip": skip,
          "subthemeId": -1,
          "take": 12,
          "themeId": -1
        }

        proxies, auth = get_proxy()
        print('1') 
        res = requests.post(endpoint, json=body, proxies=proxies, auth=auth)
        print('2') 

        if not res.ok:
            print('HTTP error:', res.status_code, res._content)
            break
        else:
            print(res._content)
            res_json = json.loads(res._content.decode('utf-8'))
            # Exit loop if no record found
            if len(res_json) == 0:
                break
            #print(res_json) 
            df = pd.json_normalize(res_json)
            all_books = all_books.append(df, ignore_index=True)
            print('Total book number:', len(all_books))
        
        skip += 12
        time.sleep(1)
        
    all_books.to_csv('/data/libreriainternacional.csv')
    print(len(all_books), 'BOOKS SAVED', )

except Exception as e:
    print('Exception', e)
    traceback.print_exc()
    all_books.to_csv('/data/libreriainternacional.csv')


1
2
b'[{"author":"","image":"/img/prod_thumbs/9788441401761.jpg","id":181101,"Theme":null,"ThemeId":0,"Subtheme":null,"isWished":false,"SquareImage":false,"Date":"","Advance":false,"ApplyPromotion":false,"PromoDescription":null,"NonBook":false,"Name":"LIBRO TIBETANO DE LOS MUERTOS"},{"author":"","image":"/img/prod_thumbs/9788883959110.jpg","id":180445,"Theme":null,"ThemeId":0,"Subtheme":null,"isWished":false,"SquareImage":false,"Date":"","Advance":false,"ApplyPromotion":false,"PromoDescription":null,"NonBook":false,"Name":"TAROT PAMELA RIDER WAITE (EX175)"},{"author":"","image":"/img/prod_thumbs/9788416407101.jpg","id":178991,"Theme":null,"ThemeId":0,"Subtheme":null,"isWished":false,"SquareImage":false,"Date":"","Advance":false,"ApplyPromotion":false,"PromoDescription":null,"NonBook":false,"Name":"GANCHILLO PARA PRINCIPIANTES"},{"author":"","image":"/img/prod_thumbs/9781594773921.jpg","id":178487,"Theme":null,"ThemeId":0,"Subtheme":null,"isWished":false,"SquareImage":false,"Date":"","A

In [167]:
def isbn_check(isbn):
    if type(isbn) == float:
        isbn = '{:.0f}'.format(isbn)
    elif type(isbn) == int:
        isbn = '{:d}'.format(isbn)
        
    # Remove dashes
    isbn = isbn.replace('-','')
    
    if len(isbn) == 10 and re.match(r'^[0-9]*$', isbn[0:9]) is not None:
        isbn = '978' + isbn[0:9] + '0'
        
    if len(isbn) == 13 and re.match(r'^[0-9]*$', isbn) is not None:
        check = 0
        for i, ch in enumerate(isbn[0:12]):
            check += int(ch) * (1 + i % 2 * 2)

        check = check % 10
        if check > 0:
            check = 10 - check

        return isbn[0:12]+str(check)
    else:
        return ''
    
proxylist = pd.read_csv('proxylist.csv', sep='\t')
current_proxy = 0

def get_proxy():
    global current_proxy
    i = current_proxy
    current_proxy = (current_proxy+1) % len(proxylist)
    
    # print('Proxy used: ', i)
    # proxies = {"http":proxylist['ip'][i]+':'+str(proxylist['port'][i]), "https":proxylist['ip'][i]+':'+str(proxylist['port'][i])}
    proxies = {"http": proxylist['ip'][i] + ':' + str(proxylist['port_http'][i])}
    auth = HTTPProxyAuth(proxylist['login'][i], proxylist['password'][i])

    return proxies, auth

In [58]:
# Function to scrap one book from www.libreriainternacional.com
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

def text_or_tag(e):
    if isinstance(e, Tag):
        return e.get_text()
    else:
        return e
        

books = pd.DataFrame()

for i in range(len(all_books)):
    endpoint = 'https://www.libreriainternacional.com/Product/Detail/%s' % all_books['id'][i]
    #endpoint = 'https://www.libreriainternacional.com/Product/Detail/%d' % i

    proxies, auth = get_proxy()
    res = requests.get(endpoint, proxies=proxies, auth=auth)

    if not res.ok:
        print('HTTP error:', res.status_code, res._content)
    else:
        soup = BeautifulSoup(res._content, 'html.parser')

        author, publisher, genre, year, title, isbn, language, description = '', '', '', '', '', '', '', ''

        details = soup.find('div', class_='product-detail-box')
        if details is not None:
            title_el = details.find('h1', class_='product-title')
            if title_el is not None:
                title = title_el.text.strip()

            author_el = details.find('span', string='Autor:')
            if author_el is not None:
                author = author_el.next_sibling.strip()

            publisher_el = details.find('span', string='Editorial:')
            if publisher_el is not None:
                publisher = publisher_el.next_sibling.strip()

            genre_el = details.find('span', string='Tema(s):')
            if genre_el is not None:
                genre = genre_el.next_sibling.strip()

            year_el = details.find('span', string='Fecha de publicación:')
            if year_el is not None:
                year = year_el.next_sibling.strip()

            isbn_el = details.find('span', string='Código:')
            if isbn_el is not None:
                isbn = isbn_el.next_sibling.strip()

            language_el = details.find('span', string='Idioma:')
            if language_el is not None:
                language = language_el.next_sibling.strip()

        description_el = soup.find('div', class_='article-container style-1')
        if description_el is not None:
            #print(description_el, '\n')
            #print('==========================================')
            head_el = description_el.find('h4', text=re.compile(r'^Autor:.*$'))
            if head_el is not None:
                #print(head_el, '\n')
                #print('==========================================')
                description = ''.join([text_or_tag(e) for e in head_el.next_siblings]).strip()

        #print(description)
        #print('==========================================')
        row = pd.DataFrame({'id': all_books['id'][i], 
                            'image': 'https://www.libreriainternacional.com' + all_books['image'][i], 
                            'author': author, 
                            'publisher': publisher, 
                            'genre': genre, 
                            'year': year, 
                            'title': title, 
                            'isbn': isbn, 
                            'language': language, 
                            'description': description}, index=[0])

        books = books.append(row, ignore_index=True)
        time.sleep(0.2)
        #print(id, image, author, publisher, genre, year, title, isbn, language, description)
        
        if (i+1) % 100 == 0:
            print(i+1, 'books collected')
            books.to_csv('datasets/libreriainternacional-details.csv')

print(i+1, 'books collected')
books.to_csv('datasets/libreriainternacional-details.csv')

books.head()

100 books collected
200 books collected
300 books collected
400 books collected
500 books collected
600 books collected
700 books collected
800 books collected
900 books collected
1000 books collected
1100 books collected
1200 books collected
1300 books collected
1400 books collected
1500 books collected
1600 books collected
1700 books collected
1800 books collected
1900 books collected
2000 books collected
2100 books collected
2200 books collected
2300 books collected
2400 books collected
2500 books collected
2600 books collected
2700 books collected
2800 books collected
2900 books collected
3000 books collected
3100 books collected
3200 books collected
3300 books collected
3400 books collected
3500 books collected
3600 books collected
3700 books collected
3800 books collected
3900 books collected
4000 books collected
4100 books collected
4200 books collected
4300 books collected
4400 books collected
4500 books collected
4600 books collected
4700 books collected
4800 books collected
4

,id,image,author,publisher,genre,year,title,isbn,language,description
0,181101,https://www.libreriainternacional.com/img/prod...,,EDAF,Espiritualidad,1900,LIBRO TIBETANO DE LOS MUERTOS,9788441401761,Español,"El Bardo-Thödol se atribuye a Padmasambhava, e..."
1,180445,https://www.libreriainternacional.com/img/prod...,,LO SCARABEO,Tarots,2014,TAROT PAMELA RIDER WAITE (EX175),9788883959110,,22 Arcanos Mayores y 56 Arcanos Menores con in...
2,178487,https://www.libreriainternacional.com/img/prod...,,INNER TRADITIONS,Tarots,2011,ORACULO MAYA,9781594773921,Español,Un instrumento práctico para aprovechar el pot...
3,178344,https://www.libreriainternacional.com/img/prod...,,GAIA EDICIONES,Nueva era,2012,CONSTELAR LA ENFERMEDAD DESDE LAS COMPRESIONES...,9788484454090,Español,Brigitte Champetier de Ribes abre un camino in...
4,176370,https://www.libreriainternacional.com/img/prod...,,HERDER,Diccionarios,1900,PROGRAMM EJERCICIOS ALEMÁN PARA HISPANOPARLANTES,9788425418594,Alemán / Español,Programm Ejercicios/Soluciones completa y desa...


In [7]:
genres = {
'Novela contemporánea': "/libros/novela-contemporanea/128000000", 
'Arte': "/libros/arte/101000000",  
'Autoayuda y Espiritualidad': "/libros/autoayuda-y-espiritualidad/102000000", 
'Ciencias Humanas': "/libros/ciencias-humanas/104000000", 
'Ciencias Políticas y Sociales': "/libros/ciencias-politicas-y-sociales/105000000",
'Ciencias': "/libros/ciencias/103000000",
'Cocina': "/libros/cocina/106000000",
'Cómics manga infantil y juvenil': "/libros/comics-manga-infantil-y-juvenil/412000000",
'Cómics': "/libros/comics/411000000", 
'Deportes y juegos': "/libros/deportes-y-juegos/108000000",
'Derecho': "/libros/derecho/109000000",
'Economía': "/libros/economia/110000000", 
'Empresa': "/libros/empresa/111000000", 
'Filología': "/libros/filologia/112000000",
'Fotografía': "/libros/fotografia/113000000",
'Guías de viaje': "/libros/guias-de-viaje/114000000",
'Historia': "/libros/historia/115000000", 
'Idiomas': "/libros/idiomas/116000000", 
'Infantil': "/libros/infantil/117000000",
'Informática': "/libros/informatica/118000000",
'Ingeniería': "/libros/ingenieria/119000000", 
'Juvenil': "/libros/juvenil/117001014", 
'Libros de Texto y Formación': "/libros/libros-de-texto-y-formacion/132000000", 
'Libros latinoamericanos': "/libros/libros-latinoamericanos/417000000", 
'Literatura': "/libros/literatura/121000000", 
'Manualidades': "/libros/manualidades/122000000",
'Medicina': "/libros/medicina/123000000", 
'Música': "/libros/musica/124000000", 
'Narrativa histórica': "/libros/narrativa-historica/125000000", 
'Novela negra': "/libros/novela-negra/126000000", 
'Oposiciones': "/libros/oposiciones/129000000",
'Psicología y Pedagogía': "/libros/psicologia-y-pedagogia/130000000", 
'Romántica y erótica': "/libros/romantica-y-erotica/416000000", 
'Salud y Dietas': "/libros/salud-y-dietas/131000000" 
}


In [4]:
# Script to read books from 
from pathlib import Path
import concurrent.futures
import os.path

def scrap_page(endpoint):
    books = []

    proxies, auth = get_proxy()
    res = requests.get(endpoint, proxies=proxies, auth=auth)

    if res.ok:
        soup = BeautifulSoup(res._content, 'html.parser')

        #author, publisher, genre, year, title, isbn, language, description = '', '', '', '', '', '', '', ''

        #books_els = soup.findAll('div', {'class': 'row', 'index': True}, partial=False)
        books_els = soup.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ['row'] and tag.get('index') is not None)

        for b in books_els:
            authors, link, isbn, image, title, description = [], '', '', '', '', ''

            title_el = b.find('a', class_='title')
            if title_el is not None:
                title = title_el.get_text().strip()
                link = 'https://www.casadellibro.com' + title_el['href']

            author_els = b.findAll('a', class_='author')
            if author_els is not None:
                authors = [a.get_text().strip() for a in author_els]

            image_el = b.find('img')
            if image_el is not None:
                image = image_el['data-src']
                isbn = Path(image).stem
                if isbn.startswith('defecto'):
                    image = ''
                    isbn = ''


            description_el = b.find('div', class_='short')
            if description_el is not None:
                description = description_el.get_text().strip()
                description = description.encode('utf-8', 'replace').decode('utf-8')

            #print(authors, link, isbn, image, title, description)

            books.append({'link': link, 
                          'image': image, 
                          'authors': ';'.join(authors), 
                          'title': title, 
                          'isbn': isbn, 
                          'description': description})
        return res, books
    elif res.status_code == 404:     
        return res, None
    else:
        return res, None
    

def scrap_topic(g):
    topic = genres[g]
    topic_name = topic.split('/')[-2]
    print(topic_name, ':', '>>>>>>>>>>>>>>>>>>>>>>>> START SCRAPPING: %s' % g)
    
    if os.path.isfile('/data/casadellibro_%s.csv' % topic_name):
        books_df = pd.read_csv('/data/casadellibro_%s.csv' % topic_name)
    else:
        books_df = pd.DataFrame()
        
    i = len(books_df) // 60 + 1
    
    errors = 0
    missing = []
    books = []
    
    while True:
        if errors >= 10:
            break
            
        endpoint = "https://www.casadellibro.com%s/p%d" % (topic, i);
        res, rows = scrap_page(endpoint)

        if res.ok:
            errors = 0 
            books.extend(rows)
        elif res.status_code == 404:     
            print(topic_name, ':', 'TOPIC completed')
            break
        else:
            print(topic_name, ':', 'HTTP error [%d] on page %s' % (res.status_code, endpoint))
            errors += 1
            missing.append(i)
        
        if i % 100 == 0:
            books_df = books_df.append(pd.DataFrame.from_dict(books), ignore_index=True)
            books_df.to_csv('/data/casadellibro_%s.csv' % topic_name)
            print(len(books_df), 'books collected')
            books = []

        i += 1
        time.sleep(0.47)
        
    books_df = books_df.append(pd.DataFrame.from_dict(books), ignore_index=True)
    books_df.to_csv('/data/casadellibro_%s.csv' % topic_name)
    print(topic_name, ':', len(books_df), 'books collected')
    print(topic_name, ':', 'Missing pages:', missing)

    books = []
    for l in range(3): 
        missing_pages[g] = [p for p in missing]
        missing = []
        
        # Quit loop if no missing pages
        if len(missing_pages[g]) == 0:
            break
            
        for i in missing_pages[g]:
            endpoint = "https://www.casadellibro.com%s/p%d" % (topic, i);
            res, rows = scrap_page(endpoint)
            if res.ok:
                books.extend(rows)
            elif res.status_code == 404:     
                break
            else:
                missing.append(i)
            
    books_df = books_df.append(pd.DataFrame.from_dict(books), ignore_index=True)
    books_df.to_csv('/data/casadellibro_%s.csv' % topic_name)
    print(topic_name, ':', len(books_df), 'books collected')
    print(topic_name, ':', 'Missing pages:', missing)

    return


def scrap_missing(g):
    topic = genres[g]
    topic_name = topic.split('/')[-2]
    print(topic_name, ':', '>>>>>>>>>>>>>>>>>>>>>>>> MISSING PAGES: %s' % g)
    print(topic_name, ':', 'Missing pages:', missing_pages[g])
    
    books = pd.read_csv('/data/casadellibro_%s.csv' % topic_name)

    for l in range(3): 
        missing = []
        
        # Quit loop if no missing pages
        if len(missing_pages[g]) == 0:
            break
            
        for i in missing_pages[g]:
            endpoint = "https://www.casadellibro.com%s/p%d" % (topic, i);
            res, rows = scrap_page(endpoint)
            if res.ok:
                books = books.append(rows, ignore_index=True)
            elif res.status_code == 404:     
                break
            else:
                missing.append(i)
                
        missing_pages[g] = missing
            
    print(topic_name, ':', len(books), 'books collected')
    books.to_csv('/data/casadellibro_%s.csv' % topic_name)
    print(topic_name, ':', 'Missing pages:', missing_pages[g])

    return
  
#missing_pages = {}

# Multy-threaded
#with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#    executor.map(thread_function, range(3))

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    executor.map(scrap_topic, [page for page in genres])
    
#with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
#    executor.map(scrap_missing, [page for page in missing_pages if len(missing_pages[page]) > 0])
#print('<<<<<<<<<<<<<<< MISSING DONE', missing_pages)

#for m in missing_pages:
#    print('\'%s\': [%s],' % (m, ','.join([str(i) for i in missing_pages[m]])))

novela-contemporanea : >>>>>>>>>>>>>>>>>>>>>>>> START SCRAPPING: Novela contemporánea


/usr/lib/python3.8/threading.py:870: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


444000 books collected
450000 books collected
456000 books collected
462000 books collected
468000 books collected
474000 books collected
480000 books collected
486000 books collected
492000 books collected
498000 books collected
504000 books collected
510000 books collected
516000 books collected
522000 books collected
528000 books collected
534000 books collected
540000 books collected
546000 books collected
552000 books collected
558000 books collected
564000 books collected
570000 books collected
576000 books collected
581958 books collected
587958 books collected
593918 books collected
599918 books collected
605918 books collected
611918 books collected
617918 books collected
623918 books collected
629918 books collected
635918 books collected
641806 books collected
647806 books collected
653806 books collected
659806 books collected
665806 books collected
671806 books collected
677806 books collected
683806 books collected
689806 books collected
695806 books collected
701806 book

In [163]:
# Consolidate all spanish books
all_books = pd.DataFrame()

for g in genres: 
    topic = genres[g]
    topic_name = topic.split('/')[-2]
    books = pd.read_csv('datasets/casadellibro_%s.csv' % topic_name, usecols = ['link','image','authors','title','isbn','description'])
    print('Read %s = %d books' % (g, len(books)))
    
    all_books = all_books.append(books, ignore_index=True, sort=False)
    
print(len(all_books))

Read Arte = 87707 books
Read Autoayuda y Espiritualidad = 36387 books
Read Ciencias Humanas = 168693 books
Read Ciencias Políticas y Sociales = 103238 books
Read Ciencias = 19303 books
Read Cocina = 18306 books
Read Cómics manga infantil y juvenil = 9852 books
Read Cómics = 38518 books
Read Deportes y juegos = 28794 books


c:\users\stark\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Read Derecho = 85679 books
Read Economía = 42142 books
Read Empresa = 45850 books
Read Filología = 154907 books
Read Fotografía = 12326 books
Read Guías de viaje = 41133 books


c:\users\stark\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Read Historia = 411001 books
Read Idiomas = 69217 books
Read Infantil = 175599 books
Read Informática = 29364 books
Read Ingeniería = 25705 books
Read Juvenil = 55809 books
Read Libros de Texto y Formación = 114991 books
Read Libros latinoamericanos = 18 books


c:\users\stark\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Read Literatura = 243657 books
Read Manualidades = 6148 books
Read Medicina = 41288 books
Read Música = 20212 books
Read Narrativa histórica = 136174 books
Read Novela contemporánea = 712049 books
Read Novela negra = 26959 books
Read Oposiciones = 28308 books
Read Psicología y Pedagogía = 83475 books
Read Romántica y erótica = 9179 books
Read Salud y Dietas = 15624 books
3097612


In [9]:
genres = {
'Idiomas': "/libros/idiomas/116000000", 
'Infantil': "/libros/infantil/117000000",
'Informática': "/libros/informatica/118000000",
'Ingeniería': "/libros/ingenieria/119000000", 
'Juvenil': "/libros/juvenil/117001014", 
'Libros de Texto y Formación': "/libros/libros-de-texto-y-formacion/132000000", 
'Libros latinoamericanos': "/libros/libros-latinoamericanos/417000000", 
'Literatura': "/libros/literatura/121000000", 
'Manualidades': "/libros/manualidades/122000000",
'Medicina': "/libros/medicina/123000000", 
'Música': "/libros/musica/124000000", 
'Narrativa histórica': "/libros/narrativa-historica/125000000", 
'Novela negra': "/libros/novela-negra/126000000", 
'Oposiciones': "/libros/oposiciones/129000000",
'Psicología y Pedagogía': "/libros/psicologia-y-pedagogia/130000000", 
'Romántica y erótica': "/libros/romantica-y-erotica/416000000", 
'Salud y Dietas': "/libros/salud-y-dietas/131000000" 
}


def isbn_check(isbn, link):
    if type(isbn) == float:
        isbn = '{:.0f}'.format(isbn)
    elif type(isbn) == int:
        isbn = '{:d}'.format(isbn)

    if type(isbn) == str and isbn[-2:] == '.0':
        isbn = isbn[:-2]

    # Restore ISBN from link
    if isbn == '':
        parts = link.split('/')
        if len(parts) >= 2:
            isbn = parts[-2]
            if not isbn.startswith('978'):
                isbn = ''

    # Remove dashes
    isbn = isbn.replace('-','')

    if len(isbn) == 10 and re.match(r'^[0-9]*$', isbn[0:9]) is not None:
        isbn = '978' + isbn[0:9] + '0'

    if len(isbn) == 13 and re.match(r'^[0-9]*$', isbn) is not None:
        check = 0
        for i, ch in enumerate(isbn[0:12]):
            check += int(ch) * (1 + i % 2 * 2)

        check = check % 10
        if check > 0:
            check = 10 - check

        return isbn[0:12]+str(check)
    else:
        return ''

#topic = genres['Derecho']
#topic = genres['Literatura']
#topic = genres['Historia']

total_count = 0

for g in genres: 
    topic = genres[g]
    topic_name = topic.split('/')[-2]
    b = pd.read_csv('/data/casadellibro_%s.csv' % topic_name, usecols = ['link','image','authors','title','isbn','description'])

    total = len(b)

    b.fillna(value='', inplace=True)
    #b.isbn = b.isbn.apply(isbn_check)
    b.isbn = b.apply(lambda x: isbn_check(x['isbn'], x['link']), axis=1)
    #no_isbn = b[b.isbn == '']
    
    b = b[b.isbn != '']
    b = b[~b.duplicated('isbn')]
    
    print(g, 'complete unique records', len(b), 100 * len(b) // total, '%')
    
    total_count += len(b)

    b.to_csv('/data/casadellibro_%s.csv' % topic_name)
    
print('TOTAL BOOKS:', total_count)

Idiomas complete unique records 59240 85 %
Infantil complete unique records 156002 88 %
Informática complete unique records 25352 86 %
Ingeniería complete unique records 22912 88 %
Juvenil complete unique records 53774 96 %
Libros de Texto y Formación complete unique records 103496 89 %
Libros latinoamericanos complete unique records 6 66 %
Literatura complete unique records 215750 88 %
Manualidades complete unique records 5448 88 %
Medicina complete unique records 34317 82 %
Música complete unique records 16001 78 %
Narrativa histórica complete unique records 124683 92 %
Novela negra complete unique records 24341 87 %
Oposiciones complete unique records 25960 92 %
Psicología y Pedagogía complete unique records 76561 90 %
Romántica y erótica complete unique records 8482 92 %
Salud y Dietas complete unique records 14421 92 %
TOTAL BOOKS: 966746


In [10]:
genres = {
'Literatura': "/libros/literatura/121000000", 
'Manualidades': "/libros/manualidades/122000000",
'Medicina': "/libros/medicina/123000000", 
'Música': "/libros/musica/124000000", 
'Narrativa histórica': "/libros/narrativa-historica/125000000", 
'Novela negra': "/libros/novela-negra/126000000", 
'Oposiciones': "/libros/oposiciones/129000000",
'Psicología y Pedagogía': "/libros/psicologia-y-pedagogia/130000000", 
'Romántica y erótica': "/libros/romantica-y-erotica/416000000", 
'Salud y Dietas': "/libros/salud-y-dietas/131000000" 
'Novela contemporánea': "/libros/novela-contemporanea/128000000", 
'Arte': "/libros/arte/101000000",  
'Autoayuda y Espiritualidad': "/libros/autoayuda-y-espiritualidad/102000000", 
'Ciencias Humanas': "/libros/ciencias-humanas/104000000", 
'Ciencias Políticas y Sociales': "/libros/ciencias-politicas-y-sociales/105000000",
'Ciencias': "/libros/ciencias/103000000",
'Cocina': "/libros/cocina/106000000",
'Cómics manga infantil y juvenil': "/libros/comics-manga-infantil-y-juvenil/412000000",
'Cómics': "/libros/comics/411000000", 
'Deportes y juegos': "/libros/deportes-y-juegos/108000000",
'Derecho': "/libros/derecho/109000000",
'Economía': "/libros/economia/110000000", 
'Empresa': "/libros/empresa/111000000", 
'Filología': "/libros/filologia/112000000",
'Fotografía': "/libros/fotografia/113000000",
'Guías de viaje': "/libros/guias-de-viaje/114000000",
'Historia': "/libros/historia/115000000", 
}


import datetime
from pathlib import Path
import concurrent.futures

def scrap_details(endpoint):
    global count
    proxies, auth = get_proxy()
    res = requests.get(endpoint, proxies=proxies, auth=auth)

    if res.ok:
        soup = BeautifulSoup(res._content, 'html.parser')

        image, pages, author, publisher, genre, subgenre, year, title, isbn, language, description = '', '', '', '', '', '', '', '', '', '', ''

        genre_anchor = soup.find('a', text = 'Escribe tu opinión')
        if genre_anchor is not None:
            genre_div = genre_anchor.findNext('div')
            if genre_div is not None:
                genre_div = genre_div.findNext('div')
                if genre_div is not None:
                    genre_a = genre_div.find('a')
                    if genre_a is not None:
                        genre = genre_a.get_text().strip()
                        # Find sub genre
                        genre_a = genre_a.findNext('a')
                        if genre_a is not None:
                            subgenre = genre_a.get_text().strip()
                            
        isbn_anchor = soup.find('meta', {'property': 'book:isbn'})
        if isbn_anchor is not None:
            isbn = isbn_anchor['content']
        
        title_el = soup.find('title')
        if title_el is not None:
            title = title_el.get_text().strip()
            if title is not None:
                title = title.split('|')[0]

        author_anchor = soup.find('meta', {'property': 'book:author'})
        if author_anchor is not None:
            author = author_anchor['content']

        year_anchor = soup.find('meta', {'property': 'book:release_date'})
        if year_anchor is not None:
            year = year_anchor['content'][:4]

        publisher_anchor = soup.find('div', text = 'Editorial:')
        if publisher_anchor is not None:
            publisher_div = publisher_anchor.findNext('div')
            if publisher_div is not None:
                publisher = publisher_div.get_text().strip()
        
        language_anchor = soup.find('div', text = 'Idioma:')
        if language_anchor is not None:
            language_div = language_anchor.findNext('div')
            if language_div is not None:
                language = language_div.get_text().strip()
            
        pages_anchor = soup.find('div', text = 'Nº de páginas:')
        if pages_anchor is not None:
            pages_div = pages_anchor.findNext('div')
            if pages_div is not None:
                pages = pages_div.get_text().strip()

        description_div = soup.find('div', class_ = 'resume-body')
        if description_div is not None:
            description = description_div.get_text().strip()
            description = description.encode('utf-8', 'replace').decode('utf-8')

        image_el = soup.find('img', class_ = 'product-image')
        if image_el is not None:
            image = image_el['src']
            name = Path(image).stem
            if name.startswith('defecto'):
                image = ''

        if isbn == '':
            isbn_anchor = soup.find('div', text = 'ISBN:')
            if isbn_anchor is not None:
                isbn_div = isbn_anchor.findNext('div')
                if isbn_div is not None:
                    isbn = isbn_div.get_text().strip()

        if year == '':
            year_anchor = soup.find('div', text = 'Año de edición:')
            if year_anchor is not None:
                year_div = year_anchor.findNext('div')
                if year_div is not None:
                    year = year_div.get_text().strip()
                    
            
        books.append({'genre': genre, 
                      'subgenre': subgenre, 
                      'authors': author, 
                      'link': endpoint, 
                      'title': title, 
                      'isbn': isbn, 
                      'image': image, 
                      'year': year, 
                      'publisher': publisher, 
                      'language': language, 
                      'description': description})
        
        print("\rBooks scanned: %d   " % count, end="")
        count += 1
        
        return res
    else:
        errors.append({'code': res.status_code, 
                      'link': endpoint})
        print('\rRequest error %d for [%s]\n' % (res.status_code, endpoint), end="")
        return res

    
i = 0
for g in genres: 
    topic = genres[g]
    topic_name = topic.split('/')[-2]

    print('\r%s %s\n' % (datetime.datetime.fromtimestamp(time.time()).strftime('%H:%M:%S'), g), end="")
     
    df_links = pd.read_csv('/data/casadellibro_%s.csv' % topic_name, usecols = ['link'])
    links = df_links['link'].tolist()
    print('\r %d books to scan\n' % len(links), end="")

    #df_books = pd.read_csv('datasets/details_casadellibro_%s.csv' % topic_name, usecols = ['genre','subgenre','authors','title','isbn','image','year','publisher','language','description'])
    #df_books= pd.DataFrame()

    books = []
    errors = []
    count = 0
    
    while True:
        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
            executor.map(scrap_details, links[i*10000:(i+1)*10000])

        time.sleep(2)
            
        print('\r %d details collected. %d %% errors\n' % (len(books), 100 * len(errors) // len(links)), end="")

        #df_add_books = pd.DataFrame.from_dict(books)
        #df_books = df_books.append(df_add_books, ignore_index=True, sort=False)
        df_books = pd.DataFrame.from_dict(books)
        df_errors = pd.DataFrame.from_dict(errors)

        df_books.to_csv('/data/details_casadellibro_%s.csv' % topic_name)    
        df_errors.to_csv('/data/errors_casadellibro_%s.csv' % topic_name)    
        #res, book = scrap_details(endpoint)

        i += 1
        if i * 10000 > len(links):
            i = 0
            break
    i = 0

01:23:31 Idiomas
 59240 books to scan
Request error 410 for [https://www.casadellibro.com/libro-trampoline-niveau-2-methode-de-francais/9782190335315/423637]
Request error 410 for [https://www.casadellibro.com/libro-comment-le-h-trouva-sa-voix-suivi-de-s-ou-deux-s----sos/9782725622798/1222898]
Request error 410 for [https://www.casadellibro.com/libro-livret-de-bd-les-miserables/9782090382976/2203851]
Request error 410 for [https://www.casadellibro.com/libro-jus-dorange-niv2-eleve--dvd/9782090384109/2502710]
Request error 410 for [https://www.casadellibro.com/libro-jus-dorange-niv1-eleve--dvd/9782090384086/2502711]
Request error 410 for [https://www.casadellibro.com/libro-le-hongrois-tout-de-suite-libro--casete/9782266076739/619699]
Request error 410 for [https://www.casadellibro.com/libro-mon-premier-dictionnaire-natha/9782092547113/2195794]
Request error 410 for [https://www.casadellibro.com/libro-sashiko-dhier-et-daujourdhu/9782299001685/2040884]
Request error 410 for [https://www.ca

Request error 404 for [https://www.casadellibro.com/libro-/9780194118286/640663]
Request error 410 for [https://www.casadellibro.com/libro-le-petit-trampoline-exercices-graphie-vocabulaire/9782090335149/586991]
Request error 410 for [https://www.casadellibro.com/libro-dictionnaire-de-larcheologie/9782221079041/423728]
Request error 410 for [https://www.casadellibro.com/libro-cid---college/9782091887593/5813206]
Request error 410 for [https://www.casadellibro.com/libro-dejouez-les-pieges/9782321011040/5534656]
Request error 410 for [https://www.casadellibro.com/libro-trouvez-le-bon-mot/9782321011026/5534790]
Request error 410 for [https://www.casadellibro.com/libro-oh-la-la-college-livre-de-l-eleve/9782090354485/1098837]
Request error 410 for [https://www.casadellibro.com/libro-sciences-techniquescom/9782090331868/1094605]
Request error 410 for [https://www.casadellibro.com/libro-methode-soda-niv1-eleve--dvd/9782090387056/1989383]
Request error 410 for [https://www.casadellibro.com/libr

Request error 404 for [https://www.casadellibro.com/libro-pratique-grammaire-a1-a2-livre--corriges/9782090389852/11331021]
 29860 details collected. 0 % errors
Request error 410 for [https://www.casadellibro.com/libro-vingt-mille-lieues-sous-les-mers--cd-audio-mp3-lectures-francai-s-facile/9782090318401/1779142]
Request error 410 for [https://www.casadellibro.com/libro-interaction-eleve-a11--dvd/9782090386998/2121042]
 39858 details collected. 0 % errors
Request error 404 for [https://www.casadellibro.com/libro-/9788493430269/1030241]
Request error 410 for [https://www.casadellibro.com/libro-grammaire-progres-francais-3e/9782090381023/7101051]
 49856 details collected. 0 % errors
Request error 410 for [https://www.casadellibro.com/libro-communication-progr-perfection/9782090380705/6398264]
 59095 details collected. 0 % errors
02:37:14 Infantil
 156002 books to scan
Request error 410 for [https://www.casadellibro.com/libro-coffret-collector-hunger-games3-vol/9782266240581/2197906]
Reque

Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977121/6574722]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977305/6585317]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1-/9781945977329/6585105]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977411/6585289]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1----/9781945977381/6585106]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977763/6750517]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977664/6750515]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1--/9781945977718/6750511]
Request error 410 for [https://www.casadellibro.com/libro-number-story-1---/9781945977749/6750507]
Request error 410 for [https://www.casadellibro.com/libro-the-number-story-1-/978194

Request error 410 for [https://www.casadellibro.com/libro-on-se-souvient-du-nom-des-assassins/9782757870730/6416196]
Request error 410 for [https://www.casadellibro.com/libro-j-ai-toujours-cette-musique-dans-la-tete/9782266282888/6416178]
Request error 410 for [https://www.casadellibro.com/libro-les-soeurs-ennemies--une-enquete-de-milo-sturgis-et-alex-delaware/9782757871201/6416184]
Request error 410 for [https://www.casadellibro.com/libro-grandes-traversees-qui-a-bu-l-eau-du-nil--raid-papou/9782757871263/6416189]
Request error 410 for [https://www.casadellibro.com/libro-la-pension-de-la-via-saffi/9782757869581/6416187]
Request error 410 for [https://www.casadellibro.com/libro-avant-que-les-ombres-s-effacent/9782757869901/6416192]
Request error 410 for [https://www.casadellibro.com/libro-le-temps-de-lilas/9782266281300/6416180]
Request error 410 for [https://www.casadellibro.com/libro-prince-de-cochinchine/9782264073198/7874858]
Request error 410 for [https://www.casadellibro.com/libro

...

Request error 410 for [https://www.casadellibro.com/libro-moxyland/9782266255226/3102127]
Request error 410 for [https://www.casadellibro.com/libro-journal-d-un-cure-de-campagne/9782266122221/1735763]
Request error 410 for [https://www.casadellibro.com/libro-sang-des-koenigsmark-t2-fils/9782266181051/1736564]
Request error 410 for [https://www.casadellibro.com/libro-odyssee-blanche/9782266162401/1737125]
Request error 410 for [https://www.casadellibro.com/libro-miroir-des-ombres/9782264046321/1748675]
Request error 410 for [https://www.casadellibro.com/libro-animaux-de-t-choupi/9782092512814/1743855]
Request error 410 for [https://www.casadellibro.com/libro-tout-maigret-t01/9782258073401/1751208]
Request error 410 for [https://www.casadellibro.com/libro-bir-hakeim-fevrier-juin-1942/9782258080126/1752237]
Request error 410 for [https://www.casadellibro.com/libro-vitamine-niveau-2-guide-pedago/9782090354744/1757925]
Request error 410 for [https://www.casadellibro.com/libro-panorama-nivea

Request error 410 for [https://www.casadellibro.com/libro-tout-simenon-t10-centenaire/9782258060517/1751070]
Request error 410 for [https://www.casadellibro.com/libro-vocabulaire-espagnol/9782266172240/1726932]
Request error 410 for [https://www.casadellibro.com/libro-sula/9782264021052/1748819]
Request error 410 for [https://www.casadellibro.com/libro-bon-anniversaire-gafi/9782092507148/1743752]
Request error 410 for [https://www.casadellibro.com/libro-alice-au-pays-des-merveilles/9782266203395/1733743]
Request error 410 for [https://www.casadellibro.com/libro-lettres-persanes/9782266196987/1737993]
Request error 410 for [https://www.casadellibro.com/libro-gafi-et-le-robot-amoureux/9782092512579/1743759]
Request error 410 for [https://www.casadellibro.com/libro-ruy-blas-a-petit-prix/9782266152204/1737922]
Request error 410 for [https://www.casadellibro.com/libro-wilt-3/9782264042453/1749104]
Request error 410 for [https://www.casadellibro.com/libro-wilt-2/9782264042446/1749110]
Reques

Request error 410 for [https://www.casadellibro.com/libro-paris-vs-new-york/9782264056306/1893017]
Request error 410 for [https://www.casadellibro.com/libro-la-grande-anthologie-de-la-fantasy/9782258058583/894354]
Request error 410 for [https://www.casadellibro.com/libro-victoria-ou-le-secret-des-fleurs/9782266220187/1994099]
Request error 410 for [https://www.casadellibro.com/libro-les-vacances-d-un-serial-killer/9782266222303/1994093]
Request error 410 for [https://www.casadellibro.com/libro-time-riders-t02-jour-predateur/9782092536872/1997341]
Request error 410 for [https://www.casadellibro.com/libro-mars-la-verte/9782266128490/909756]
Request error 410 for [https://www.casadellibro.com/libro-le-coup-du-pere-francois/9782265075832/914211]
Request error 410 for [https://www.casadellibro.com/libro-collations/9782232122057/890733]
Request error 410 for [https://www.casadellibro.com/libro-le-tropique-et-le-nord-l-hopital-de-leningrad-et-autres-nouvelle-s/9782707139900/906687]
Request er

KeyboardInterrupt: 

Books scanned: 49536   

In [15]:
#df_books = pd.read_csv('datasets/errors_casadellibro_literatura-1.csv', usecols = ['genre','subgenre','authors','link','title','isbn','image','year','publisher','language','description'])
df_books = pd.read_csv('datasets/errors_casadellibro_literatura-1.csv', usecols = ['code','link'])
df_add = pd.DataFrame.from_dict(errors)

df_books = df_books.append(df_add, ignore_index=True)

df_books.to_csv('datasets/errors_casadellibro_literatura.csv')    
